In [1]:
import os
import sys
import time as time

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
current_file_path = os.getcwd()
src_path = os.path.abspath(os.path.join(current_file_path, "../src"))
print(f"src path: {src_path}")
sys.path.insert(0, src_path)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

src path: /ehome/xinyi/Xinyi_GNN_aorta/src


device(type='cuda')

In [2]:
from utils import aorta_3D_info, Normalizer_ts

processed_path = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_single/raw'
outputpath     = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_single'
data_label = ['y', 'pos', "stmdist"]

data_information = aorta_3D_info(path=processed_path, labels=data_label, output_path=outputpath)

csv already exists, loading data...
data load done.


In [3]:
# from data.dataset import Poisson_2d_Dataset
# from data.data_utils import poisson_info

# root = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_single'
# data_label = ['temperature', 'source']
# data_information = poisson_info(path=root+'/raw', labels=data_label, output_path=root)

In [4]:
data_information.info

,max,min,mean,std
label,,,,
pressure,3.398470,-7.650037,1.278087,1.304617
wssx,0.021710,-0.046168,-0.002896,0.006476
wssy,0.010297,-0.074118,-0.008381,0.009542
wssz,0.079113,-0.036755,0.006116,0.017247
x,38.824947,-17.211851,15.181118,13.173093
y,46.847960,-48.988758,-3.707316,24.354790
z,82.896675,-5.897548,42.835144,18.339022
distance,177.199800,0.000000,67.495740,45.188190


In [5]:
# root = '/home/yongqi/GNNField/poisson'
# dataset = Poisson_2d_Dataset(root)

In [6]:
# train_size = 400
# test_size = 100
# train_dataset = dataset[:train_size]
# test_dataset = dataset[train_size:train_size+test_size]

In [7]:
import numpy as np

# Extracting output ranges from the data information for data normalization

output_range = data_information.info.values[0].reshape(1, -1) # pressure!!!!!!!!!!!!!
output_range = output_range.astype(np.float32)

# Step 1: 提取 x, y, z 三行的数据（索引为 4, 5, 6）
xyz_rows = data_information.info.values[4:7].astype(np.float32)

# Step 2: 提取 mean 和 std 列（索引 2 和 3）
xyz_mean = xyz_rows[:, 2]  # shape = (3,)
xyz_std = xyz_rows[:, 3]   # shape = (3,)

# Step 3: 计算 mean 和 std 的平均值
shared_mean = np.mean(xyz_mean)
shared_std = np.mean(xyz_std)

# Step 4: 构造新的 mean/std 参数，用于 x/y/z 和其他维度（比如 distance）拼接
# x/y/z 使用统一的 mean/std，distance 用原来的
distance_row = data_information.info.values[7].astype(np.float32)  # index 7 是 distance

# 构造 input normalization 参数，按列顺序排列
# shape = (2, 4) → [mean_row, std_row]
input_mean = np.array([shared_mean, shared_mean, shared_mean, distance_row[2]])
input_std = np.array([shared_std, shared_std, shared_std, distance_row[3]])
input_params = np.vstack([input_mean, input_std])  # shape = (2, 4)

In [8]:
# input_normalizer = Normalizer_ts(method='ms',params=input_range[:,2:].T)
input_normalizer = Normalizer_ts(method='ms', params=input_params)
output_normalizer = Normalizer_ts(method="ms", params=output_range[:,2:].T)

In [9]:
# from torch_geometric.loader import DataLoader

# train_loader = DataLoader(train_dataset, batch_size = 10)
# test_loader = DataLoader(test_dataset, batch_size = 1)

In [10]:
from Bench_Heat.heat_sampling_suk import HeatSamplingCluster
from dataset import Aorta_3d_Dataset

# Multiscale radius graph
ratios = [1., 0.4, 0.1]
radii = [0.042, 0.06, 0.8] # if artery_type == 'single' 
# else [0.022, 0.04, 0.1]

trans = HeatSamplingCluster(ratios, radii)


root = '/ehome/xinyi/Xinyi_GNN_aorta/data/toy_single'
dataset = Aorta_3d_Dataset(root,'cpu', label='pressure', pre_transform=trans,input_normalizer=input_normalizer,output_normalizer=output_normalizer)

In [11]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split

total = len(dataset)
train_size = int(0.9 * total)
valid_size = int(0.1 * total)
test_size  = total - train_size

train_dataset, test_dataset= random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(0))
_, valid_dataset = random_split(train_dataset, [train_size-valid_size, valid_size], generator=torch.Generator().manual_seed(1))

train_loader = DataLoader(train_dataset, batch_size = 4, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size = 1)
test_loader = DataLoader(test_dataset, batch_size = 1)

In [2]:
from Bench_Heat.single_scale.gnn_model import MGN
from torch.optim.lr_scheduler import ExponentialLR

model = MGN()
model.to(device)

lr = 0.001

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=0.001)
schedule = ExponentialLR(optimizer,.999)

optim_para = {
            'epoch': 0,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss':1,
            }

MGN (232441 trainable parameters)


In [16]:
from tqdm import tqdm
from train import train, valid

import torch
import matplotlib.pyplot as plt
import os

# ✅ checkpoint 路径
ckpt_dir = 'checkpoints_toy'
os.makedirs(ckpt_dir, exist_ok=True)

# ✅ 图像保存路径
fig_dir = 'figures_toy'
os.makedirs(fig_dir, exist_ok=True)

# ✅ 初始化日志
train_epoch_error = []
val_epoch_error = []
test_epoch_error = []
check_idx = []

epochs = 1000

# ✅ 开始训练
for epoch in tqdm(range(epochs)):
    train_loss = train(model, device, train_loader, optimizer, criterion, scheduler=schedule)
    train_epoch_error.append(train_loss)

    if (epoch  % 100) == 0:
        val_loss = valid(model, device, valid_loader, criterion)
        test_loss = valid(model, device, test_loader, criterion)

        val_epoch_error.append(val_loss)
        test_epoch_error.append(test_loss)
        check_idx.append(epoch)

        # ✅ 保存 checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'test_loss': test_loss
        }, os.path.join(ckpt_dir, f'checkpoint_epoch_{epoch}.pt'))

        # ✅ 画图 1：训练损失
        fig1, ax1 = plt.subplots(figsize=(8, 5))
        ax1.plot(
            range(len(train_epoch_error)),
            torch.stack(train_epoch_error).detach().cpu().numpy(),
            color='C0', label='Train Loss'
        )
        ax1.set_yscale('log')
        ax1.set_title('Train Loss Curve')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        fig1.tight_layout()
        fig1.savefig(os.path.join(fig_dir, f"train_loss_epoch_{epoch}.png"))
        plt.close(fig1)

        # ✅ 画图 2：Val/Test 损失（每 100 epoch）
        fig2, ax2 = plt.subplots(figsize=(8, 5))
        ax2.plot(
            check_idx,
            torch.stack(val_epoch_error).detach().cpu().numpy(),
            color='C1', label='Validation Loss'
        )
        ax2.plot(
            check_idx,
            torch.stack(test_epoch_error).detach().cpu().numpy(),
            color='C2', label='Test Loss'
        )
        ax2.set_yscale('log')
        ax2.set_title('Validation & Test Loss (every 100 epochs)')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.legend()
        fig2.tight_layout()
        fig2.savefig(os.path.join(fig_dir, f"val_test_loss_epoch_{epoch}.png"))
        plt.close(fig2)


100%|██████████| 1000/1000 [10:33<00:00,  1.58it/s]


In [14]:
# def train(model, device, train_loader, optimizer, criterion, scheduler=None):
#     report_loss = 0
#     model.train()
#     for i, batch in enumerate(train_loader):
#         optimizer.zero_grad()
#         batch = batch.to(device)
#         label = batch.y
#         output = model(batch)
#         loss = criterion(output, label)  # masked optimisation
#         report_loss += loss.item()
#         loss.backward()
#         # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, error_if_nonfinite=True)
#         optimizer.step()
#         if scheduler != None:
#             scheduler.step()
            
#     return report_loss/len(train_loader)

# def valid(model, device, val_loader, criterion):
#     model.eval()
#     valid_loss = 0
#     with torch.no_grad():
#         for _, batch in enumerate(val_loader):
#             batch = batch.to(device)
#             label = batch.y
#             output = model(batch)
#             loss = criterion(output, label)
#             valid_loss += loss.item()

#     return valid_loss/len(val_loader)


# from tqdm import tqdm
# import matplotlib.pyplot as plt
# #start training 
# train_epoch_error = []
# val_epoch_error = []
# test_error = []
# check_idx = []
# epochs = 1000

# for epoch in tqdm(range(epochs)):
#     train_epoch_error.append(train(model, device, train_loader, optimizer,criterion))
#     test_error.append(valid(model, device, test_loader,criterion))
#     check_idx.append(epoch+1)


#     if (epoch+1)%500==0 and epoch != 0:    
#         fig, ax = plt.subplots(figsize=(10,8))
#         er1 = torch.tensor(train_epoch_error)
#         er3 = torch.tensor(test_error)
#         ax.plot(er1[1:],color = 'C0',linestyle='solid',linewidth=1,alpha=1,label='L_train')
#         ax.plot(er3,color = 'C2',linestyle='solid',linewidth=1,alpha=1,label='L_test')
#         ax.set_yscale('log')
#         ax.set_title('train_loss_curve')
#         ax.set_xlabel('epochs')
#         ax.set_ylabel('loss')
#         ax.legend()
#         plt.show()

In [ ]:
from MP_Model.metrics import Metrics

with torch.no_grad():

    Tabulate the evaluation metrics
    table = Metrics([test_loader]).statistics(model, torch.device('cpu'))
    print(table)

ModuleNotFoundError: No module named 'MP_Model'

In [ ]:
# import torch
# import pyvista as pv
# import matplotlib.pyplot as plt
# from os.path import join as osj
# from tqdm import tqdm

# def generate_output(
#     model,
#     device,
#     output_loader,
#     raw_mesh_path,
#     tag,
#     save_mesh_path,
#     mesh_name="mesh",
#     # output_normalizer=None,
# ):
#     for _, batch in enumerate(tqdm(output_loader)):
#         batch = batch.to(device)
#         with torch.no_grad():
#             label = batch.y
#             output = model(batch)
#             idx = (batch.idx)[0]

#             mesh = pv.read(osj(raw_mesh_path, "single_artery{:04d}.vtu".format(idx)))
#             mesh.point_data[tag + "_prediction"] = output.detach().cpu()
#             mesh.point_data[tag + "_label"] = label.detach().cpu()
#             mesh.point_data[tag + "_error"] = (
#                 mesh.point_data[tag + "_prediction"] - mesh.point_data[tag + "_label"]
#             )
#             mesh.save(osj(save_mesh_path, mesh_name + "_{:d}.vtu".format(idx)))


# # generate_output(model,'cpu',train_loader,'aorta/raw','wss','results/meshes','mesh_wss_1',output_normalizer)

In [ ]:
from MP_Model.metrics import Metrics

with torch.no_grad():

    # Tabulate the evaluation metrics
    table = Metrics([test_loader]).statistics(model, torch.device('cpu'))
    print(table)